# Multi-armed Bandits Algorithms

This notebook explains and reproduces the most popular multi-armed bandits (MAB) algorithms:
- epsilon-greedy
- UCB
- Thompson Sampling

### What?
MAB is a classic reinforcement learning problem for balancing the trade-off between exploration vs exploitation.
Essentially, MAB formulates various kinds of sequential decision-making problems; thus, it is applicable to a wide range of scenarios.

### Example
In a casino, a gambler plays a slot machine that has `K` pulling arms (the story if there are `K` slot machines with one arm). 
How to leave the casino without being bankrupt? Or even with some earnings?

##### Rules
1. Each arm has a reward distribution that is initially unknown to the gambler.
2. Each round, the gambler can only play one arm and only observe the reward of that very arm.

##### Goal
Maximize the cumulative reward before he bankrupts or the casino closes.